# Projeto 1 - Ciência dos Dados

Nome: Guilherme Martins

Nome: Giovana Cassoni

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [184]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [185]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/giovanaandrade/Documents/Insper/Ciência dos Dados/Projeto 1/Projeto1_C.Dados-2


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [186]:
filename = 'Air Fryer.xlsx'

In [187]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,air fryer pra canhoto,1
1,@vanessapeitos @daizenbvc o franky conseguiria...,0
2,adoro poder aplicar meus conhecimentos de enge...,1
3,"@leilodecornohu1 ""amigo estamos falando de buc...",0
4,se eu n mandar isso aqui na dm de alguma esqui...,0


In [188]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@poetatulio @tatyene_mendes isso é uma xota ou...,0
1,"comprei uma air fryer p minha mãe, o bagulho caro",0
2,usando air fryer como se não houvesse o amanhã,1
3,minha mãe comprou uma air fryer e já ama mais ...,1
4,repita comigo: churrasco na air fryer não existe!,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [189]:
import re 

def cleanup(text):
    punctuation = '[@!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [190]:
def cleanup2(text):
    paragrafo = '\n'
    pattern = re.compile(paragrafo)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [191]:
def cleanup3(text):
    punctuation = '\/'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [192]:
trainR = train.loc[train['Relevância'] == 1,:]
trainN = train.loc[train['Relevância'] == 0,:]

In [193]:
trainR = trainR.Treinamento.apply(str)
trainR = trainR.to_list()

In [194]:
trainN = trainN.Treinamento.apply(str)
trainN = trainN.to_list()

In [195]:
stringR = " "
for i in trainR:
    stringR = stringR + i + ' '
stringR = stringR.lower()
stringR = stringR.replace('air fryer', 'PRODUTO ')
stringR = cleanup3(cleanup2(cleanup(stringR)))
palavrasR = stringR.split()

In [196]:
stringN = " "
for i in trainN:
    stringN = stringN + i + ' '
stringN = stringN.lower()
stringN = stringN.replace('air fryer', 'PRODUTO ')
stringN = cleanup3(cleanup2(cleanup(stringN)))
palavrasN = stringN.split()

In [197]:
serie_tweets_relevantes = pd.Series(palavrasR)
serie_tweets_relevantes.head(10)

0          PRODUTO
1              pra
2          canhoto
3            adoro
4            poder
5          aplicar
6             meus
7    conhecimentos
8               de
9       engenharia
dtype: object

In [198]:
serie_tweets_nao_relevantes = pd.Series(palavrasN)
serie_tweets_nao_relevantes.head(10)

0    vanessapeitos
1        daizenbvc
2                o
3           franky
4      conseguiria
5          colocar
6              uma
7       fritadeira
8          PRODUTO
9               no
dtype: object

In [199]:
tabela_tweets_relevantes = serie_tweets_relevantes.value_counts()
tabela_tweets_relevantes

PRODUTO       174
de             96
uma            72
e              71
na             61
             ... 
ano             1
parte           1
vendo           1
ttbemerguy      1
muitooo         1
Length: 1020, dtype: int64

In [200]:
tabela_tweets_nao_relevantes = serie_tweets_nao_relevantes.value_counts()
tabela_tweets_nao_relevantes

PRODUTO      129
de            69
uma           57
a             55
que           46
            ... 
utiliza        1
horizonte      1
adaptada       1
vamos          1
vale           1
Length: 857, dtype: int64

In [201]:
tabela_tweets_relevantes_relativa = serie_tweets_relevantes.value_counts(True)
tabela_tweets_relevantes_relativa

PRODUTO       0.064877
de            0.035794
uma           0.026846
e             0.026473
na            0.022744
                ...   
ano           0.000373
parte         0.000373
vendo         0.000373
ttbemerguy    0.000373
muitooo       0.000373
Length: 1020, dtype: float64

In [202]:
tabela_tweets_nao_relevantes_relativa = serie_tweets_nao_relevantes.value_counts(True)
tabela_tweets_nao_relevantes_relativa

PRODUTO      0.060450
de           0.032334
uma          0.026710
a            0.025773
que          0.021556
               ...   
utiliza      0.000469
horizonte    0.000469
adaptada     0.000469
vamos        0.000469
vale         0.000469
Length: 857, dtype: float64

In [203]:
train = train.Treinamento.apply(str)
train = train.to_list()

In [204]:
lista_dos_tweets = []
lista_tweet = []
for tweet in train:
  lista_tweet.append(tweet)
  lista_dos_tweets.append(lista_tweet)
  lista_tweet = []
lista_dos_tweets

[['air fryer pra canhoto'],
 ['@vanessapeitos @daizenbvc o franky conseguiria colocar uma fritadeira air fryer no pau'],
 ['adoro poder aplicar meus conhecimentos de engenharia eletrica (acabei de arrumar a air fryer)'],
 ['@leilodecornohu1 "amigo estamos falando de buceta, não de uma air fryer " kkkkkkk'],
 ['se eu n mandar isso aqui na dm de alguma esquisita ainda hoje eu sou uma air fryer https://t.co/f2fjtqqhbi'],
 ['@nataliabridi viu que a mesma mulher colocava ate chocolate na 🐸 ate ele derreter pro amado poder descer? hahahahaha\né a buceta air fryer'],
 ['fui fazer um bolinho na air fryer, esborrou\nganhei bolinho de biscoitinhos de chocolate'],
 ['descobri que batata doce na air fryer é ruim'],
 ['@renataabritto a mulher era uma air fryer, uma air fryer'],
 ['eu queria uma air fryer'],
 ['a fritadeira elétrica nell fit é um eletroportátil que não pode faltar na sua cozinha. o produto proporciona uma alimentação mais saudável, pois não utiliza óleo/air fryer em seu processo de 

In [205]:
string = " "
for i in train:
    string = string + i + ' '
string = string.lower()
string = string.replace('air fryer', 'PRODUTO ')
string = cleanup3(cleanup2(cleanup(string)))
palavras = string.split()

In [206]:
i=0
y = []
while i < 300:
   x = lista_dos_tweets[i]
   x = x[0].lower()
   x = x.replace('air fryer', 'PRODUTO ')
   x = cleanup3(cleanup2(cleanup(x)))
   x=x.split()
   y.append(x)
   i+=1
y


[['PRODUTO', 'pra', 'canhoto'],
 ['vanessapeitos',
  'daizenbvc',
  'o',
  'franky',
  'conseguiria',
  'colocar',
  'uma',
  'fritadeira',
  'PRODUTO',
  'no',
  'pau'],
 ['adoro',
  'poder',
  'aplicar',
  'meus',
  'conhecimentos',
  'de',
  'engenharia',
  'eletrica',
  'acabei',
  'de',
  'arrumar',
  'a',
  'PRODUTO'],
 ['leilodecornohu1',
  'amigo',
  'estamos',
  'falando',
  'de',
  'buceta',
  'não',
  'de',
  'uma',
  'PRODUTO',
  'kkkkkkk'],
 ['se',
  'eu',
  'n',
  'mandar',
  'isso',
  'aqui',
  'na',
  'dm',
  'de',
  'alguma',
  'esquisita',
  'ainda',
  'hoje',
  'eu',
  'sou',
  'uma',
  'PRODUTO',
  'https',
  'tco',
  'f2fjtqqhbi'],
 ['nataliabridi',
  'viu',
  'que',
  'a',
  'mesma',
  'mulher',
  'colocava',
  'ate',
  'chocolate',
  'na',
  '🐸',
  'ate',
  'ele',
  'derreter',
  'pro',
  'amado',
  'poder',
  'descer',
  'hahahahaha',
  'é',
  'a',
  'buceta',
  'PRODUTO'],
 ['fui',
  'fazer',
  'um',
  'bolinho',
  'na',
  'PRODUTO',
  'esborrou',
  'ganhei',
 

In [207]:
probR = len(serie_tweets_relevantes)/len(palavras)
probN = len(serie_tweets_nao_relevantes)/len(palavras)
probR, probN

(0.5568936877076412, 0.4431063122923588)

In [208]:
i = 0
listaR=[]
while i < 300:
    p = y[i]
    probFraseDadoR = 1
    for n in p:
        probFraseDadoR = probFraseDadoR * (tabela_tweets_relevantes_relativa[i]+1)
    listaR.append(probFraseDadoR)
    i+=1

In [209]:
i = 0
listaN=[]
while i < 300:
    p = y[i]
    probFraseDadoN = 1
    for n in p:
        probFraseDadoN = probFraseDadoN * tabela_tweets_nao_relevantes_relativa[i]
    listaN.append(probFraseDadoN)
    i+=1

In [210]:
i = 0
listax=[]
while i < 300:
    probRDadoFrase = listaR[i] * probR
    listax.append(probRDadoFrase)
    i+=1

In [211]:
i = 0
listan=[]
while i < 300:
    probNDadoFrase = listaN[i] * probN
    listan.append(probNDadoFrase)
    i+=1

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [212]:
test = test.Teste.apply(str)
test = test.to_list()

In [217]:
lista_dos_tweets2 = []
lista_tweet2 = []
for tweet in test:
  lista_tweet2.append(tweet)
  lista_dos_tweets2.append(lista_tweet2)
  lista_tweet2 = []

In [216]:
i=0
y = []
while i < 200:
   x = lista_dos_tweets2[i]
   x = x[0].lower()
   x = x.replace('air fryer', 'PRODUTO ')
   x = cleanup3(cleanup2(cleanup(x)))
   x=x.split()
   y.append(x)
   i+=1

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**